<a href="https://colab.research.google.com/github/and-rgr/contradiction_and_entailment/blob/main/dce_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [3]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import BertTokenizer, TFBertModel
from transformers import TFAutoModel,AutoTokenizer

In [4]:
pd.set_option('display.expand_frame_repr', False)

# os.environ["WANDB_API_KEY"] = "0"

In [5]:
# set up the TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU

print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [40]:
# load data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

# reduce data - optional
train = train.iloc[:100]
test = test.iloc[:50]

print("train data shape: ", train.shape)
print("test data shape: ", test.shape)

na_train = len(train) - len(train.dropna())
na_test = len(test) - len(test.dropna())

print("check for missing rows:", "\n\ttrain data:", na_train, ", test data:", na_test)

train data shape:  (100, 6)
test data shape:  (50, 5)
check for missing rows: 
	train data: 0 , test data: 0


In [94]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# learning_rate = 1e-5
# batch_size = 32
# warmup = 600
# max_seq_length = 128
# num_train_epochs = 3.0

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# def encode_sentence(s):
#    tokens = list(tokenizer.tokenize(s))
#    tokens.append('[SEP]')
#    return tokenizer.convert_tokens_to_ids(tokens)

In [71]:
# def bert_encode(hypotheses, premises, tokenizer):
#     num_examples = len(hypotheses)
#     sentence1 = tf.ragged.constant([encode_sentence(s) for s in np.array(hypotheses)])
#     sentence2 = tf.ragged.constant([encode_sentence(s) for s in np.array(premises)])

#     cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
#     input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)
#     input_mask = tf.ones_like(input_word_ids).to_tensor()

#     type_cls = tf.zeros_like(cls)
#     type_s1 = tf.zeros_like(sentence1)
#     type_s2 = tf.ones_like(sentence2)
#     input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=-1).to_tensor()

#     inputs = {
#         'input_word_ids': input_word_ids.to_tensor(),
#         'input_mask': input_mask,
#         'input_type_ids': input_type_ids}
#     return inputs, sentence1

In [72]:
# train_input, sentence1train = bert_encode(train.premise.values, train.hypothesis.values, tokenizer)
# test_input, sentence1test = bert_encode(test.premise.values, test.hypothesis.values, tokenizer)

In [98]:
# train_data = list(train.loc[:,["premise","hypothesis"]]))

a = train["premise"].to_list()
b = train["hypothesis"].to_list()
train_data = [a,b]

# tf_batch = tokenizer(
#     [a,b],
#     padding=True,
#     truncation=True,
#     max_length=512,
#     return_tensors="tf"
# )
# TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]


# train_enc=tokenizer.batch_encode_plus(train[['premise','hypothesis']].values.tolist(),padding='max_length',max_length=100,truncation=True,return_attention_mask=True)
# test_enc=tokenizer.batch_encode_plus(test[['premise','hypothesis']].values.tolist(),padding='max_length',max_length=100,truncation=True,return_attention_mask=True)

# is .batch_encode_plus needed here?
tf_batch = tokenizer.batch_encode_plus(
    train[['premise','hypothesis']].values.tolist(),
    padding=True,
    truncation=True,
    max_length=100,
    return_tensors="tf"
)

In [101]:
# why training = True?
tf_outputs = tf_model(tf_batch, training = True)

In [90]:
# If you have labels, you can provide them to the model, it will return a tuple with the loss and the final activations.
# tf_outputs = tf_model(tf_batch, labels = tf.constant([1, 0]))

In [86]:
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

In [87]:
type(tf_predictions)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
tf_predictions

In [ ]:
with strategy.scope():
    # embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    # output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=tf_predictions)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

In [ ]:
model.fit(train_input, train.label.values, epochs = 2, verbose = 1, batch_size = None, validation_split = 0.2)